# 3. Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты

In [1]:
n_zev = 10
n = 34

n_contr_zev = 4
n_contr = 16

In [2]:
import numpy as np
import pandas as pd
import scipy
from statsmodels.stats.proportion import proportion_confint

In [3]:
ZEV = proportion_confint(n_zev, n, alpha=0.05, method='normal')
CONTR = proportion_confint(n_contr_zev, n_contr, alpha=0.05, method='normal')

In [4]:
ZEV, CONTR

((0.14096118119136916, 0.44727411292627794),
 (0.03782767486071778, 0.4621723251392822))

In [5]:
z = scipy.stats.norm.ppf(1 - 0.05 / 2.)

p1 = float(n_zev/ n)
p2 = float(n_contr_zev/n_contr)

left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n + p2 * (1 - p2)/ n_contr)
right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n + p2 * (1 - p2)/ n_contr)
print("Доверительный интервал равен:", left_boundary, right_boundary)

Доверительный интервал равен: -0.2175577216559601 0.3057930157736072


In [6]:
n1 = n
n2 = n_contr

p1 = float(n_zev) / n1
p2 = float(n_contr_zev) / n2 
P = float(p1*n1 + p2*n2) / (n1 + n2)
Z_stat = (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))
print("Z-статистика равна:", Z_stat)

Z-статистика равна: 0.32410186177608225


In [7]:
def proportions_diff_z_test(P, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(P)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(P)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(P)

In [8]:
print('Достигаемый уровень значимости при альтернативе заразительности зевоты равен', 
      round(proportions_diff_z_test(Z_stat, alternative = 'greater'), 4))

Достигаемый уровень значимости при альтернативе заразительности зевоты равен 0.3729


# 4. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры 

In [9]:
data = pd.read_table('banknotes.txt')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [10]:
from sklearn.model_selection import train_test_split
data_train, data_test, y_train, y_test = train_test_split(data.drop(['real'], axis = 1), data.real, 
                                                          test_size=0.25, random_state=1)

In [11]:
from sklearn.linear_model import LogisticRegression
LR13 = LogisticRegression().fit(data_train[['X1', 'X2', 'X3']], y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [12]:
LR46 = LogisticRegression().fit(data_train[['X4', 'X5', 'X6']], y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [13]:
pred13 = LR13.predict(data_test[['X1', 'X2', 'X3']])
pred46 = LR46.predict(data_test[['X4', 'X5', 'X6']])

In [14]:
1 - sum(pred46 == y_test)/50, 1 - sum(pred13 == y_test)/50

(0.020000000000000018, 0.19999999999999996)

In [15]:
n13 = 50
n46 = 50

p13 = float(sum(pred13 == y_test)) / n13
p46 = float(sum(pred46 == y_test)) / n46
P = float(p13*n13 + p46*n46) / (n13 + n46)
Z_stat = (p13 - p46) / np.sqrt(P * (1 - P) * (1. / n13 + 1. / n46))
print("Z-статистика равна:", Z_stat)

Z-статистика равна: -2.8764124744526685


In [16]:
print('Достигаемый уровень значимости гипотезы, что доли ошибочных предсказаний двух классификаторов равны', 
      round(proportions_diff_z_test(Z_stat, alternative = 'two-sided'), 4))

Достигаемый уровень значимости гипотезы, что доли ошибочных предсказаний двух классификаторов равны 0.004


In [17]:
from statsmodels.stats.weightstats import *
stats.ttest_rel(pred13, pred46)

Ttest_relResult(statistic=-1.5275252316519468, pvalue=0.13306042398416879)

In [18]:
4 * (10**-3)

0.004

# 5. В предыдущей задаче посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? 

In [28]:
from statsmodels.stats.weightstats import DescrStatsW
DescrStatsW(abs(1-pred13) - abs(1 - pred46)).tconfint_mean()

(-0.031557580382220424, 0.23155758038222043)

In [29]:
round(-0.031557580382220424, 4)

-0.0316

# 6. Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? 

In [23]:
(541.4-525)/(100/10)

1.6399999999999977

In [26]:
from scipy.stats import norm
round(1-norm.cdf((541.4-525)/(100/10)), 4)

0.0505

# 7. средний балл 100 выпускников которых равен 541.5. 

In [27]:
from scipy.stats import norm
round(1-norm.cdf((541.5-525)/(100/10)), 4)

0.0495